In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import skimage.transform as trf
from utiles import *

In [ ]:
import os
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms

In [ ]:
target = data_loader("data/lfwcrop_grey",shape=(64, 64))

In [ ]:
n_ratio = .1
data = target + n_ratio*np.random.normal(0, 1, target.shape)
data = np.clip(data, a_max=1, a_min=0)

x_train, x_test, y_train, y_test = train_test_split( data, target, test_size=.25, random_state=15)

In [ ]:
x_test_r, x_test_dl = to_tensor( x_test, b_size=50 )
y_test_r, y_test_dl = to_tensor( y_test, b_size=50 )

In [ ]:
plot_n_faces(x_train, y_train, 2, h_size=4)

## Model

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self, img_shape=(28, 28)):
        super(Autoencoder, self).__init__()
        self.out_s = np.array(img_shape)/8
        self.c1 = nn.Conv2d( 1, 10, 3, padding=1, stride=1 )
        self.c2 = nn.Conv2d( 10, 20, 3, padding=1, stride=1 )
        self.c3 = nn.Conv2d( 20, 40, 3, padding=1, stride=1 )

        self.ct1 = nn.ConvTranspose2d( 40, 20, 3, padding=1, stride=2 )
        self.ct2 = nn.ConvTranspose2d( 20, 10, 3, padding=1, stride=2 )
        self.ct3 = nn.ConvTranspose2d( 10, 1, 3, padding=1, stride=2 )
        self.convf = nn.Conv2d( 1, 1, 3, padding=1, stride=1 )

    def encode( self, x ):
        x = nn.ReLU(True)( self.c1( x ) )
        x = nn.MaxPool2d( kernel_size=2, stride=2 )( x )
        x = nn.ReLU(True)( self.c2( x ) )
        x = nn.MaxPool2d( kernel_size=2, stride=2 )( x )
        x = nn.ReLU(True)( self.c3( x ) )
        x = nn.MaxPool2d( kernel_size=2, stride=2 )( x )
        return x

    def decode( self, code ):
#         out_s = np.array(code.size())[-2:]*2 
        out_s = self.out_s *2
        x = self.ct1( code, output_size=out_s )
        x = nn.ReLU( True )( x )
        out_s = out_s*2
        x = self.ct2( x, output_size=out_s )
        x = nn.ReLU( True )( x )
        out_s = out_s*2
        x = self.ct3( x, output_size=out_s )
        x = self.convf( x )
        x = nn.Tanh()( x )
        return x
    
    def forward(self, x):
        code = self.encode( x )
        rec = self.decode( code )
        return rec

In [ ]:
model = Autoencoder(img_shape=(64, 64))
model.double()

In [ ]:
x_train_raw, x_train_dl = to_tensor( x_train, b_size=50 )
y_train_raw, y_train_dl = to_tensor( y_train, b_size=50 )

In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam( model.parameters(), lr=0.003 )
n_epochs = 5
for e in range( n_epochs ):
    for d, t in zip(x_train_dl, y_train_dl):
        optimizer.zero_grad()
        img = d[0]
        img = Variable(img)

        trg = t[0]
        trg = Variable(trg)

        output = model( img )
        loss = criterion(output, trg)
        loss.backward()
        optimizer.step()

    print("epoch [{}/{}], loss:{:.4f}".format(e + 1, n_epochs, loss.item()))

In [ ]:
torch.save(model.state_dict(), "pesos/pytorch_dae_conv.h5")

In [ ]:
model.load_state_dict( torch.load("pesos/pytorch_dae_conv.h5") )

In [ ]:
d_data = next( iter(x_test_dl) )

In [ ]:
predicted = model( Variable(d_data[0])).data.numpy()

ddata = x_test[:50]
ttarget = y_test[:50]
predicted = predicted.reshape( predicted.shape[0], 64, 64 )
ddata.shape, ttarget.shape, predicted.shape

In [ ]:
plot_n_faces(ddata, ttarget, 5, predicted)

In [ ]:
from skimage.filters import median
from skimage.morphology import selem
s = selem.square(3)
i = np.random.randint( 50 )
img = median( x_test[i], s )
f = plt.figure()
ax = f.add_subplot(1,2,1)
ax.imshow(x_test[i], cmap=plt.cm.gray)
ax = f.add_subplot(1,2,2)
ax.imshow(img, cmap=plt.cm.gray)
plt.show()